## Add available processors

In [1]:
nprocs()

1

In [2]:
nprocs() == 1 && addprocs()

4-element Array{Int64,1}:
 2
 3
 4
 5

In [3]:
workers()

4-element Array{Int64,1}:
 2
 3
 4
 5

## Remote Calls and References

In [4]:
ref = @spawn sin(π/5)

Future(2,1,6,Nullable{Any}())

In [5]:
fetch(ref)

0.5877852522924731

In [6]:
n=10
refs = Array(Future,n)
for i = 1:n
    refs[i] = @spawn sin(i)
end

In [7]:
refs

10-element Array{Future,1}:
 Future(3,1,8,#NULL) 
 Future(4,1,9,#NULL) 
 Future(5,1,10,#NULL)
 Future(2,1,11,#NULL)
 Future(3,1,12,#NULL)
 Future(4,1,13,#NULL)
 Future(5,1,14,#NULL)
 Future(2,1,15,#NULL)
 Future(3,1,16,#NULL)
 Future(4,1,17,#NULL)

In [8]:
reduce(+,map(fetch,refs))

1.4111883712180104

In [9]:
res=@parallel (vcat) for i=1:n
    sin(i)
end

10-element Array{Float64,1}:
  0.841471
  0.909297
  0.14112 
 -0.756802
 -0.958924
 -0.279415
  0.656987
  0.989358
  0.412118
 -0.544021

In [10]:
reduce(+,res)

1.4111883712180104

In [11]:
res=@parallel (vcat) for i = 1:10
    println((i,sin(i)))
    [i sin(i)]
end

	From worker 4:	(7,0.6569865987187891)
	From worker 4:	(8,0.9893582466233818)
	From worker 5:	(9,0.4121184852417566)
	From worker 5:	(10,-0.5440211108893698)
	From worker 2:	(1,0.8414709848078965)
	From worker 2:	(2,0.9092974268256817)
	From worker 2:	(3,0.1411200080598672)
	From worker 3:	(4,-0.7568024953079282)
	From worker 3:	(5,-0.9589242746631385)
	From worker 3:	(6,-0.27941549819892586)


10×2 Array{Float64,2}:
  1.0   0.841471
  2.0   0.909297
  3.0   0.14112 
  4.0  -0.756802
  5.0  -0.958924
  6.0  -0.279415
  7.0   0.656987
  8.0   0.989358
  9.0   0.412118
 10.0  -0.544021

In [12]:
sum(res[:,2])

1.4111883712180104

## Monte-Carlo Simulation to estimate $\pi$

In [13]:
#==========================#
# monte-carlo simulation
# π r^2 / 4 r^2 = s/n 
#==========================#


@everywhere function isInside() 
    x = rand()
    y = rand()
    x^2 + y^2 < 1 ? 1 : 0
end;

@everywhere function ppi(n)
    s=@parallel (+) for i = 1:n
        isInside()
    end
    4s/n
end;


In [14]:
@time ppi(10^9)

  7.235441 seconds (137.84 k allocations: 5.872 MB)


## Distributed Arrays

In [15]:
using DistributedArrays

a = round(10*rand(20,20))

20×20 Array{Float64,2}:
 8.0  2.0   1.0  1.0   2.0   8.0   8.0  …   3.0   6.0   0.0  1.0   7.0  6.0
 9.0  8.0   3.0  9.0   1.0   7.0   4.0      9.0   8.0   8.0  6.0   5.0  4.0
 1.0  8.0   5.0  9.0   9.0   2.0   0.0      9.0   3.0   5.0  2.0  10.0  1.0
 1.0  9.0  10.0  8.0   4.0   7.0   7.0      7.0  10.0   4.0  8.0   3.0  5.0
 9.0  5.0   6.0  6.0   4.0   4.0   2.0      1.0   3.0   0.0  5.0   1.0  9.0
 1.0  3.0   9.0  4.0   2.0   3.0   7.0  …   0.0   7.0   7.0  7.0   9.0  5.0
 4.0  3.0   9.0  2.0   2.0   5.0   9.0      9.0   0.0   3.0  7.0   6.0  4.0
 4.0  2.0   3.0  9.0   0.0   9.0   8.0      3.0   7.0  10.0  4.0   9.0  8.0
 6.0  9.0   3.0  5.0   4.0   6.0   3.0     10.0   5.0   6.0  2.0   5.0  7.0
 8.0  8.0   5.0  1.0   2.0  10.0   2.0      3.0   9.0   5.0  6.0   2.0  3.0
 7.0  1.0  10.0  8.0   9.0   0.0   2.0  …   0.0   8.0   0.0  7.0   8.0  8.0
 8.0  6.0   1.0  7.0   0.0   8.0   7.0     10.0   5.0   5.0  6.0   9.0  6.0
 2.0  2.0  10.0  3.0   9.0   9.0   9.0      8.0   5.0   1.0  2.0

In [16]:
map(x->myid(),a)

20×20 Array{Int64,2}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1 

In [17]:
da = distribute(a)

20×20 DistributedArrays.DArray{Float64,2,Array{Float64,2}}:
 8.0  2.0   1.0  1.0   2.0   8.0   8.0  …   3.0   6.0   0.0  1.0   7.0  6.0
 9.0  8.0   3.0  9.0   1.0   7.0   4.0      9.0   8.0   8.0  6.0   5.0  4.0
 1.0  8.0   5.0  9.0   9.0   2.0   0.0      9.0   3.0   5.0  2.0  10.0  1.0
 1.0  9.0  10.0  8.0   4.0   7.0   7.0      7.0  10.0   4.0  8.0   3.0  5.0
 9.0  5.0   6.0  6.0   4.0   4.0   2.0      1.0   3.0   0.0  5.0   1.0  9.0
 1.0  3.0   9.0  4.0   2.0   3.0   7.0  …   0.0   7.0   7.0  7.0   9.0  5.0
 4.0  3.0   9.0  2.0   2.0   5.0   9.0      9.0   0.0   3.0  7.0   6.0  4.0
 4.0  2.0   3.0  9.0   0.0   9.0   8.0      3.0   7.0  10.0  4.0   9.0  8.0
 6.0  9.0   3.0  5.0   4.0   6.0   3.0     10.0   5.0   6.0  2.0   5.0  7.0
 8.0  8.0   5.0  1.0   2.0  10.0   2.0      3.0   9.0   5.0  6.0   2.0  3.0
 7.0  1.0  10.0  8.0   9.0   0.0   2.0  …   0.0   8.0   0.0  7.0   8.0  8.0
 8.0  6.0   1.0  7.0   0.0   8.0   7.0     10.0   5.0   5.0  6.0   9.0  6.0
 2.0  2.0  10.0  3.0   9.0  

In [18]:
map(x->myid(),da)

20×20 DistributedArrays.DArray{Int64,2,Array{Int64,2}}:
 2  2  2  2  2  2  2  2  2  2  4  4  4  4  4  4  4  4  4  4
 2  2  2  2  2  2  2  2  2  2  4  4  4  4  4  4  4  4  4  4
 2  2  2  2  2  2  2  2  2  2  4  4  4  4  4  4  4  4  4  4
 2  2  2  2  2  2  2  2  2  2  4  4  4  4  4  4  4  4  4  4
 2  2  2  2  2  2  2  2  2  2  4  4  4  4  4  4  4  4  4  4
 2  2  2  2  2  2  2  2  2  2  4  4  4  4  4  4  4  4  4  4
 2  2  2  2  2  2  2  2  2  2  4  4  4  4  4  4  4  4  4  4
 2  2  2  2  2  2  2  2  2  2  4  4  4  4  4  4  4  4  4  4
 2  2  2  2  2  2  2  2  2  2  4  4  4  4  4  4  4  4  4  4
 2  2  2  2  2  2  2  2  2  2  4  4  4  4  4  4  4  4  4  4
 3  3  3  3  3  3  3  3  3  3  5  5  5  5  5  5  5  5  5  5
 3  3  3  3  3  3  3  3  3  3  5  5  5  5  5  5  5  5  5  5
 3  3  3  3  3  3  3  3  3  3  5  5  5  5  5  5  5  5  5  5
 3  3  3  3  3  3  3  3  3  3  5  5  5  5  5  5  5  5  5  5
 3  3  3  3  3  3  3  3  3  3  5  5  5  5  5  5  5  5  5  5
 3  3  3  3  3  3  3  3  3  3  5  5  5  5  5

In [19]:
da.pids

2×2 Array{Int64,2}:
 2  4
 3  5

In [20]:
[@spawnat p sum(localpart(da)) for p in da.pids]

2×2 Array{Future,2}:
 Future(2,1,3384,#NULL)  Future(4,1,3386,#NULL)
 Future(3,1,3385,#NULL)  Future(5,1,3387,#NULL)

In [21]:
map(fetch,[@spawnat p sum(localpart(da)) for p in da.pids])

2×2 Array{Float64,2}:
 504.0  525.0
 513.0  516.0

In [22]:
reduce(+,map(fetch,[@spawnat p sum(localpart(da)) for p in da.pids]))

2058.0

In [2]:
@which sum(10)

sum(a) at reduce.jl:235

## Cross-validation in parallel

In [24]:
using RDatasets
@everywhere using RDatasets
using DecisionTree
@everywhere using DecisionTree

@everywhere function irisAcc() 
    iris = dataset("datasets", "iris")
    features = convert(Array, iris[:, 1:4]);
    labels = convert(Array, iris[:, 5]);
    model = build_forest(labels, features, 2, 10, 0.5, 6);
    accuracy = nfoldCV_forest(labels, features, 2, 10, 2, 0.5);
    mean(accuracy)
end


In [25]:
n=30
@time begin 
    s=0
    for i=1:n
        s += irisAcc()
    end
    print("Overall Acc:",s/n*100.0)
end


Fold 1
Classes:  

3×3 Array{Int64,2}:
 21   0   0
  1  18   1
  0   7  27

Any["setosa","versicolor","virginica"]
Matrix:   

3×3 Array{Int64,2}:
 29   0   0
  3  26   1
  0   1  15


Accuracy: 0.88
Kappa:    0.8181083265966048

Fold 2
Classes:  Any["setosa","versicolor","virginica"]
Matrix:   
Accuracy: 0.9333333333333333
Kappa:    0.8967226659322501

Mean Accuracy: 0.9066666666666667


...

3×3 Array{Int64,2}:
 20   0   0
  0  26   4
  0   0  25


Fold 2
Classes:  Any["setosa","versicolor","virginica"]
Matrix:   

3×3 Array{Int64,2}:
 26   0   0
  0  21   4
  0   0  24

3×3 Array{Int64,2}:
 24   0   0
  6  18   1
  0   4  22


Accuracy: 0.9466666666666667
Kappa:    0.9193548387096774

Mean Accuracy: 0.9199999999999999

Fold 1
Classes:  Any["setosa","versicolor","virginica"]
Matrix:   
Accuracy: 0.9466666666666667
Kappa:    0.9200426439232411

Fold 2
Classes:  Any["setosa","versicolor","virginica"]
Matrix:   
Accuracy: 0.8533333333333334
Kappa:    0.780409901517168

Mean Accuracy: 0.9
Overall Acc:92.73333333333336 14.748917 seconds (5.45 M allocations: 238.855 MB, 0.85% gc time)


In [26]:
n=30
@time begin 
    s=@parallel (+) for i=1:n
        irisAcc()
    end
    print("Overall Acc:",s/n*100.0)
end

	From worker 3:	
	From worker 3:	Fold 1
	From worker 2:	
	From worker 2:	Fold 1
	From worker 4:	
	From worker 4:	Fold 1
	From worker 5:	
	From worker 5:	Fold 1
	From worker 3:	Classes:  Any["setosa","versicolor","virginica"]
	From worker 2:	Classes:  Any["setosa","versicolor","virginica"]
	From worker 4:	Classes:  Any["setosa","versicolor","virginica"]
	From worker 5:	Classes:  Any["setosa","versicolor","virginica"]
	From worker 2:	Matrix:   3×3 Array{Int64,2}:
	From worker 3:	Matrix:   3×3 Array{Int64,2}:
	From worker 4:	Matrix:   3×3 Array{Int64,2}:
	From worker 5:	Matrix:   3×3 Array{Int64,2}:
	From worker 2:	 22   0   0
	From worker 2:	  1  26   1
	From worker 2:	  0   3  22
	From worker 2:	Accuracy: 0.9333333333333333
	From worker 2:	Kappa:    0.8995176848874599
	From worker 3:	 23   3   0
	From worker 3:	  0  24   2
	From worker 3:	  0   5  18
	From worker 3:	Accuracy: 0.8666666666666667
	From worker 3:	Kappa:    0.7991967871485944
	From worker 4:	 26   0   0
	From worker 4:	  0 